In [18]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib, os


wine = pd.read_csv("winequality-red-selected-missing.csv")

In [ ]:
def impute_and_train(imputer, name):
    print(f"\n🔎 Testing imputer: {name}")
    

    wine_imputed = pd.DataFrame(imputer.fit_transform(wine), columns=wine.columns)
    wine_imputed["good_quality"] = (wine_imputed["quality"] >= 7).astype(int)

 
    X = wine_imputed.drop(columns=["quality", "good_quality"])
    y = wine_imputed["good_quality"]

  
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

 
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, stratify=y
    )


    model = RandomForestClassifier(random_state=42, n_estimators=200, max_depth=12)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    return model, scaler


In [20]:
knn_model, knn_scaler = impute_and_train(KNNImputer(n_neighbors=5), "KNN")
iter_model, iter_scaler = impute_and_train(IterativeImputer(random_state=42), "Iterative")



🔎 Testing imputer: KNN
Accuracy: 0.934375
              precision    recall  f1-score   support

           0       0.94      0.99      0.96       277
           1       0.89      0.58      0.70        43

    accuracy                           0.93       320
   macro avg       0.92      0.79      0.83       320
weighted avg       0.93      0.93      0.93       320


🔎 Testing imputer: Iterative
Accuracy: 0.934375
              precision    recall  f1-score   support

           0       0.94      0.99      0.96       277
           1       0.87      0.60      0.71        43

    accuracy                           0.93       320
   macro avg       0.90      0.80      0.84       320
weighted avg       0.93      0.93      0.93       320



In [21]:
final_model, final_scaler = iter_model, iter_scaler


In [22]:
os.makedirs("artifacts", exist_ok=True)
joblib.dump(final_model, "artifacts/wine_quality_model.pkl")
joblib.dump(final_scaler, "artifacts/scaler.pkl")
print("✅ Best model + scaler saved!")

✅ Best model + scaler saved!
